In [1]:
# prepare the workspace
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# read the data
artworks = pd.read_table("beautified_boxes.tsv")

In [ ]:
# cleaning and preparing the data
"""
Things still to do:
label all of the remaining rows in the data set in the branch "mock data", then commit, push, merge into main and pull changes into this branch ("data_preparation"), because the data set is not up to date yet
empty user ratings will be interpreted as nan by python -> handle this! I would say filling with average does not make sense, but a missing user rating definitely influenced my decision. Maybe fill with a new label, 
because I actually kinda handled it like that. If it had no label, chances were slimmer to get approval.
Then have a look at the columns and so on. In this branch, the columns are a bit silly, but this may get resolved by pulling the real data set. Still, check it out when it comes to loading the data.
Later on, when I use the data for the ML algorithm, I do not need all of the columns. For example, I do not actually need the ID, I think. I should think about dropping it here or I just keep it in the data set and 
later just select the needed columns.
When just loading the data, I 
"""

In [ ]:
# split the data into training and test sets
# remember that 650/1000 are labeled yet. Either label remaining ones or shorten the set
# training data  is for training the algorithm 
# test set is for evaluating the algorithm
# both must be strictly separated

# features are type, district, environment, countArtists, experience, replaced, content, user rating
# label is "approved"
# select features and label e.g. like this 
# features = data[["type", "district", "..."]]
# label = data["approved"]
# or as in the example: features is called X and label is called y
# They did it like that:
# X = fruits[["mass", "width", "height"]]
# y = fruits["fruit_label"]
# They then split the data like this: 
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 
# note that random state is for setting seed and this is necessary because they work for a large audience that wants to reproduce the result. 
# I may not need this parameter, depending on if I want to get a new result each time or not.
# Alright, to reliably assess, you even NEED to have a look at multiple splits!

In [ ]:
# visualisations
# to see range of values, outliers and so on (or actually less in my example, because the values are categorical lol)
# try to see if a potential algorithm is likely to be able to classify the data -> a well defined cluster should be visible

# good plots to visualise the data: 
# 1) feature-pair plot
# their code:
from matplotlib import cm
cmap = cm.get_cmap("gnuplot")
scatter = pd.scatter_matrix(X_train, c = y_train, marker = "o", s = 40, hist_kwds = {"bins":15}, figsize = (12, 12), cmap = cmap)

In [ ]:
# MACHINE LEARNING ALGORITHM LEARNING, OPTIONS, DECISION MAKING AND BRAINSTORM
# THIS BRANCH AND FILE IS FOR DATA PREPARATION, WRITE THE ACTUAL ML PART IN IT'S OWN BRANCH AND FILE
# THIS CELL IS JUST TO TAKE NOTES AND SO ON 
"""
ML algorithm options
-k-nearest neighbors: instance or memory based learning
Would likely work for my data set
Likely pretty easy: k=1 (one nearest neighbor)
It needs: a distance metric, k (how many nearest neighbors), weighting function on the neighbor points (not all neigh. have same influence), method for aggregating classes of neighbor points (how to combine influence and decide)
Distance: scikit uses euclidian distance per default
Neighbors: e.g. 5 (odd -> no tie -> no weighting necessary)
Weighting: not necessary, when k/2=0.5
Aggregation: Majority vote
Use-case/code in the video: about knn

Assess the performance of the algorithm later on:
e.g. compute accuracy of the classifier -> score method
Also (if I will actually use knn,) try different values for k and then assess accuracy
"""